In [1]:
import pandas as pd

from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.models import WheelZoomTool
import pandas as pd

from bokeh.layouts import column
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, CustomJS, Slider



timeframes = ['1min','5min','15min','30min','1h']

df = pd.read_csv('https://gist.githubusercontent.com/algostrat/e01326407e2b09fdd71d9ce11db714d3/raw/7fb2b8f265e84693de9b3b981ea4d86a19e24ba1/eurusd.csv')
df = df.rename(columns={'RateDateTime':'date'})

df["date"] = pd.to_datetime(df["date"])

inc = df.close > df.open
dec = df.open > df.close
w = 12*60*60*1 # half day in ms
w = pd.Timedelta('5min')*.75

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

p = figure(
    x_axis_type="datetime", 
    tools=TOOLS, plot_width=1000, 
    sizing_mode="stretch_width"
)

source = ColumnDataSource(data=df)

callback = CustomJS(args=dict(source=source), code="""
        const f = cb_obj.value;
                 console.log(f);
                """
    )

slider = Slider(start=1, end=(len(timeframes) -1) , value=1, step=1, title="Time frame")
slider.js_on_change('value', callback)

p.toolbar.active_scroll = p.select_one(WheelZoomTool)
p.segment(df.date, df.high, df.date, df.low, color="black")
p.vbar(df.date[inc], w, df.open[inc], df.close[inc], fill_color="#D5E1DD", line_color="black")
p.vbar(df.date[dec], w, df.open[dec], df.close[dec], fill_color="#F2583E", line_color="black")
p.toolbar.logo = None

layout = column( p, slider)

output_notebook()
show(layout )

Loading BokehJS ...